In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


# Load the dataset
df = pd.read_csv("C:\\Users\\akhilkrishna\\OneDrive\\Desktop\\mlearning.csv")
df1 =  pd.read_csv("C:\\Users\\akhilkrishna\\OneDrive\\Desktop\\mlearning.csv")


Pred_College = pd.DataFrame(df)



C:\Users\akhilkrishna\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
Pred_College['institute_type'] = [0 if x == 'IIT' else 1 for x in Pred_College['institute_type']]
Pred_College['institute_type'].unique()


array([0, 1], dtype=int64)

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Pred_College['quota'] = le.fit_transform(Pred_College['quota'])
Pred_College['quota'].unique()

array([0, 3, 6, 1, 2, 4, 5])

In [4]:
Pred_College['program_name'] = le.fit_transform(Pred_College['program_name'])
Pred_College['program_name'].unique()

array([  0,  28,  31,  32,  42,  47,  51,  52,  64,  66,  67,  98, 101,
       107, 108, 109,  14,  24,  48,  95, 126, 129,   1,   2,   4,   7,
        37,  46,  54,  58,  59,  72,  79,  80,  83,  86, 114,  87, 104,
       110, 116, 118, 122, 127,  20,  45,  90,  97,  18,  19,  65, 117,
         6,  13,  57,  74,  75, 124,  29,  61,  68,  91,  15,  17,  26,
        36,  53,  56,  76,  92, 105, 121,   5,  62,  70, 112, 115, 119,
        33,  34,  99, 100, 120,  35,  49, 103, 111,  10,  71, 102, 106,
         3,  60,  81,  88,   8,  16,  44,  11,  55,  50,  69,  89,  21,
         9,  38, 113,  22,  43,  63, 128,  30,  94,  25,  39,  84, 125,
        41,  93,  82,  78, 123,  23,  12,  77,  73,  27,  85,  96,  40])

In [5]:
from sklearn.preprocessing import LabelEncoder

Pred_College['institute_short'] = le.fit_transform(Pred_College['institute_short'])
Pred_College['institute_short'].unique()


array([ 4,  5, 15, 14, 16, 20,  9, 11, 10,  0, 19,  1,  3, 17,  7, 21, 13,
       22,  2,  6,  8, 12, 18, 53, 51, 52, 50, 42, 25, 34, 33, 27, 24, 28,
       23, 29, 30, 31, 32, 39, 36, 43, 41, 44, 45, 47, 26, 35, 37, 38, 40,
       46, 48, 49])

In [6]:
Pred_College['program_duration'] = le.fit_transform(Pred_College['program_duration'])
Pred_College['program_duration'].unique()


array([0, 1])

In [7]:
Pred_College['degree_short'] = le.fit_transform(Pred_College['degree_short'])
Pred_College['degree_short'].unique()

array([ 4,  7,  5, 11,  0, 10,  1,  2,  6, 12,  3,  9,  8])

In [8]:
Pred_College['pool'] = [0 if x == 'Gender-Neutral' else 1 for x in Pred_College['pool']]
Pred_College['pool'].unique()


array([0, 1], dtype=int64)

In [9]:
Pred_College['category']= le.fit_transform(Pred_College['category'])
Pred_College['category'].unique()

array([0, 4, 6, 8, 3, 5, 7, 9, 1, 2])

In [10]:
# Split the dataset into features (X) and target (y)
X = df.drop([ 'id','opening_rank', 'closing_rank','institute_type'], axis=1)
y_opening = df['opening_rank']
y_closing = df['closing_rank']


In [11]:
from sklearn.model_selection import KFold
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize KFold with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create empty lists to store the RMSE values for each fold
rmse_opening = []
rmse_closing = []

# Loop through the folds
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_opening_train, y_opening_test = y_opening.iloc[train_index], y_opening.iloc[test_index]
    y_closing_train, y_closing_test = y_closing.iloc[train_index], y_closing.iloc[test_index]
    
    # Initialize BaggingRegressor for opening ranks
    model_opening = BaggingRegressor(base_estimator=DecisionTreeRegressor(),
                                     n_estimators=10,
                                     random_state=42)
    
    # Fit the model on the training data
    model_opening.fit(X_train, y_opening_train)
    
    # Predict the opening ranks on the test data
    y_opening_pred = model_opening.predict(X_test)
    
    # Calculate the RMSE for opening ranks and append to the list
    rmse_opening.append(np.sqrt(mean_squared_error(y_opening_test, y_opening_pred)))
    
    # Initialize BaggingRegressor for closing ranks
    model_closing = BaggingRegressor(base_estimator=DecisionTreeRegressor(),
                                     n_estimators=10,
                                     random_state=42)
    
    # Fit the model on the training data for closing ranks
    model_closing.fit(X_train, y_closing_train)
    
    # Predict the closing ranks on the test data
    y_closing_pred = model_closing.predict(X_test)
    
    # Calculate the RMSE for closing ranks and append to the list
    rmse_closing.append(np.sqrt(mean_squared_error(y_closing_test, y_closing_pred)))

# Calculate the mean RMSE for opening and closing ranks
mean_rmse_opening = np.mean(rmse_opening)
mean_rmse_closing = np.mean(rmse_closing)

print(f'Mean RMSE for Opening Ranks: {mean_rmse_opening}')
print(f'Mean RMSE for Closing Ranks: {mean_rmse_closing}')


Mean RMSE for Opening Ranks: 6440.790217661691
Mean RMSE for Closing Ranks: 8777.613500692141


In [12]:
from sklearn.metrics import r2_score

r2_opening = r2_score(y_opening_test, y_opening_pred)
print('R2 score for opening rank:', r2_opening)


R2 score for opening rank: 0.9343400666172916


In [13]:
from sklearn.metrics import r2_score

r2_closing = r2_score(y_closing_test, y_closing_pred)
print('R2 score for opening rank:', r2_closing)


R2 score for opening rank: 0.9230886478543477
